In [8]:
import os

In [9]:
# Function to load documents from a directory
def load_documents_from_directory(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(
                os.path.join(directory_path, filename), "r", encoding="utf-8"
            ) as file:
                documents.append({"id": filename, "text": file.read()})
    return documents

In [10]:
# Function to split text into chunks
def split_text(text, chunk_size=200, chunk_overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    return chunks

In [11]:
# Load documents from the directory
directory_path = "./pubmed_summaries"
documents = load_documents_from_directory(directory_path)

In [12]:
# Split documents into chunks
chunked_documents = []
for doc in documents:
    chunks = split_text(doc["text"])
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{doc['id']}_chunk{i+1}", "text": chunk})

In [1]:
import chromadb
import numpy as np
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

c:\Users\22213\OneDrive\Desktop\ragKoyl\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Load the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Wrap it in a function Chroma can understand
class SentenceTransformerEmbeddingFunction:
    def __init__(self, model):
        self.model = model

    def __call__(self, input):
        return self.model.encode(input).tolist()

# Initialize the wrapped function
embedding_function = SentenceTransformerEmbeddingFunction(embedding_model)

# Initialize Chroma with persistent storage
chroma_client = chromadb.PersistentClient(path="chroma_persistent_storage")

# Create or get the collection with embedding function
collection = chroma_client.get_or_create_collection(
    name="medical_chunks",
    embedding_function=embedding_function
)


In [13]:
# Upsert into Chroma 
for doc in chunked_documents:
    collection.add(
        ids=[doc["id"]],
        documents=[doc["text"]],
    )

In [14]:
# Query Function 
def query_documents(question, n_results=2):
    results = collection.query(query_texts=[question], n_results=n_results)
    relevant_chunks = [doc for sublist in results["documents"] for doc in sublist]
    return relevant_chunks

In [15]:
import json
import requests

In [20]:
URL = 'http://localhost:11434/api/generate'
HEADERS = {
        'Content-Type': 'application/json'
    }

# Local-LLM Placeholder 
def generate_response(question):
    context = query_documents(question)

    data = {
        'model': 'deepseek-r1:1.5b',
        'prompt': f"You are a helpful doctor. I want you to help me figure out nutitions that i need to take accoding to my current symptoms.Use this context just as reference to answer:\n\n{context}\n\nQuestion: {question}",
        'stream': False
    }

    res = requests.post(URL, headers=HEADERS, data=json.dumps(data))

    if res.status_code == 200:
        res_text = res.text
        data = json.loads(res_text)
        actual_res = data['response']
        return actual_res
    
    return 'Some error occured!'

In [21]:
question = "What are the nutrition recommendations for managing anxiety?"
answer = generate_response(question)

print("\n=== Final Answer ===")
print(answer)


=== Final Answer ===
<think>
Alright, so I need to figure out the nutrition recommendations for managing anxiety based on the given context. Let me start by carefully reading the provided information to understand what's being discussed.

The first part says that gammaCore is a treatment option for CPTHA with significant improvements in anxiety levels. The results show that gammaCore might be promising because of these improvements, especially when combined with other treatments like a noninvasive approach.

Then, there's a section about the study: it mentions VAS (visual analog scale), SAS (self-rating anxiety scale) were recorded before and after treatment, with 68 patients included. The data shows that gammaCore improved anxiety levels over time. 

Now, focusing on nutrition recommendations for anxiety management. I know that nutrition can affect various health aspects, including heart health, which is often linked to anxiety in older adults. Maybe a diet rich in fruits, vegetables